## Neuro File Transfer (.ps1)

This should be kept and run in the Compute Canada folder where Oddity txt and edat2 files are backed up and sorted (Currently fishingowl). It will send new files (24hours) to Compute Canada.

*Remember to replace with actual path of your scratch folder*

General

NeuropsychBattery_Auto_Sync.ps1

In [ ]:
$currentDate = Get-Date

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/FinalOddity/"

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
    if ($?) {
        $transferredCount++
        $transferredDirectories += $dir.Name
        
        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
    } else {
        Write-Warning "Failed to transfer directory: $dir"
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}